In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix

In [ ]:
data = pd.read_csv('/diabetes.csv')

In [ ]:
X_data = data.drop('Outcome',axis = 1)
Y_data = data['Outcome']

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_data,Y_data,test_size = 0.2,random_state = 42)

In [ ]:
scaler =  StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#PERCEPTRON LIBRARY IMPLEMENTATION

In [ ]:
perceptron_model = Perceptron(max_iter = 2000, tol = 1e-3, random_state = 4)
perceptron_model.fit(X_train_scaled,y_train)
y_pred_perceptron = perceptron_model.predict(X_test_scaled)

In [ ]:
accuracy = accuracy_score(y_test,y_pred_perceptron)
precision = precision_score(y_test,y_pred_perceptron)
recall = recall_score(y_test,y_pred_perceptron)
f1 = f1_score(y_test,y_pred_perceptron)

In [ ]:
print(f"Accuracy of Perceptron Model is: {accuracy}")
print(f"Precision of Perceptron Model is: {precision}")
print(f"Recall of Perceptron Model is: {recall}")
print(f"F1 Score of Perceptron Model is: {f1}")

Accuracy of Perceptron Model is: 0.7662337662337663
Precision of Perceptron Model is: 0.6507936507936508
Recall of Perceptron Model is: 0.7454545454545455
F1 Score of Perceptron Model is: 0.6949152542372882


#PERCEPTRON CUSTOM IMPLEMENTATION

In [ ]:
class SingleLayerPerceptron:
  def __init__(self,my_weights,my_bias,learning_rate = 0.05,max_iter = 2000):
    self.weights = my_weights
    self.bias = my_bias
    self.learning_rate = learning_rate
    self.max_iter = max_iter

  def fit(self,X,y):
    X = np.array(X)
    y = np.array(y)
    y = np.where(y == 0, -1, y)
    for _ in range(self.max_iter):
      for idx,x_i in enumerate(X):
        linear_output = np.dot(x_i,self.weights) + self.bias
        y_predicted = 1 if linear_output > 0 else -1

        if(y_predicted != y[idx]):
          self.weights+=self.learning_rate*y[idx]*x_i
          self.bias+=self.learning_rate*y[idx]

  def predict(self, X_test):
    X_test = np.array(X_test)
    linear_prediction = np.dot(X_test, self.weights) + self.bias
    linear_prediction = np.where(linear_prediction > 0, 1, -1)

    return linear_prediction

  @staticmethod
  def metrics(y,y_pred):
    y = np.array(y)
    y_pred = np.array(y_pred)

    tp = np.sum((y_pred == 1) & (y == 1))
    tn = np.sum((y_pred == -1) & (y == -1))
    fp = np.sum((y_pred == 1) & (y == -1))
    fn = np.sum((y_pred == -1) & (y == 1))

    accuracy = (tp + tn) / len(y)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return accuracy, precision, recall, f1_score


In [ ]:
np.random.seed(30)
w = np.random.rand(X_train.shape[1])
b = np.random.rand()

In [ ]:
w

array([0.64414354, 0.38074849, 0.66304791, 0.16365073])

In [ ]:
b

0.9626078136743187

In [ ]:
custom_model = SingleLayerPerceptron(w,b)

custom_model.fit(X_train_scaled,y_train)

In [ ]:
custom_model.weights

array([0.08976902, 0.04398389, 0.13984575, 0.03377206])

In [ ]:
custom_model.bias

-0.03739218632568167

In [ ]:
y_pred = custom_model.predict(X_test_scaled)
y_test = np.where(y_test == 0, -1, y_test)
accuracy, precision, recall, f1_score_custom = custom_model.metrics(y_pred,y_test)
print(f"Accuracy of Custom Perceptron model is {accuracy}")
print(f"Precision of Custom Perceptron model is {precision}")
print(f"Recall of Custom Perceptron model is {recall}")
print(f"F1 Score of Custom Perceptron model is {f1_score_custom}")

Accuracy of Custom Perceptron model is 0.7077922077922078
Precision of Custom Perceptron model is 0.7090909090909091
Recall of Custom Perceptron model is 0.5735294117647058
F1 Score of Custom Perceptron model is 0.6341463414634145


In [ ]:
with open('perceptron_model.pkl','wb') as file:
  pickle.dump(perceptron_model,file)

In [ ]:
with open('custom_perceptron_model.pkl','wb') as file:
  pickle.dump(custom_model,file)

In [ ]:
scaler.mean_

array([120.85504886,  81.43811075,  31.98338762,  32.90716612])

In [ ]:
scaler.scale_

array([ 32.00895893, 116.14014299,   7.73431907,  11.49406506])

K-Fold cross verification:-

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=42) # Using 5 folds for this example

In [ ]:
accuracy_scores_library = []
precision_scores_library = []
recall_scores_library = []
f1_scores_library = []

for train_index, test_index in kfold.split(X_data):
    X_train, X_test = X_data.iloc[train_index], X_data.iloc[test_index]
    y_train, y_test = Y_data.iloc[train_index], Y_data.iloc[test_index]

    # Scale data within each fold
    scaler = StandardScaler()  # Create a new scaler for each fold
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    perceptron_model = Perceptron(max_iter=2000, tol=1e-3, random_state=4)
    perceptron_model.fit(X_train_scaled, y_train)

    y_pred = perceptron_model.predict(X_test_scaled)

    accuracy_scores_library.append(accuracy_score(y_test, y_pred))
    precision_scores_library.append(precision_score(y_test, y_pred))
    recall_scores_library.append(recall_score(y_test, y_pred))
    f1_scores_library.append(f1_score(y_test, y_pred))

# Calculate and print average metrics:
print("Library Perceptron - Average Metrics:")
print(f"Accuracy: {np.mean(accuracy_scores_library)}")
print(f"Precision: {np.mean(precision_scores_library)}")
print(f"Recall: {np.mean(recall_scores_library)}")
print(f"F1 Score: {np.mean(f1_scores_library)}")

Library Perceptron - Average Metrics:
Accuracy: 0.7201171377641966
Precision: 0.5419148936170213
Recall: 0.4142080511889155
F1 Score: 0.44442554529694334


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Performance metrics for Naive bayes is better since Perceptron models prefer linearly separable data.